### Spark?  
Apache Spark는 '빅데이터 처리'를 용이하게 하기 위한 플랫폼으로, 기존 Hadoop이 Disk I/O 기반이었던것에 비하여, In-memory 방식으로 동작  
현재 대중적인 구조는 Hadoop의 YARN 위에 Spark를 얹고, 실시간성이 필요한 데이터를 Spark로 처리하는 방식이다.  
### Spark Architecture  
Spark application은 하나의 드라이버 프로세스와 다수의 excutor 프로세스로 구성된다.
1. 드라이버 프로세스  
- 클러스터 노드 중 하나에서(master) 실행  
- Spark Application에서 정보 유지관리, 배포, 스케쥴링 등 심장과 같은 역할  
- 생명주기 관리  
- Spark Session이라는 인터페이스를 통하여 JVM에서 명시적으로 작성하지 않고 Python, R 등 여러 언어를 실행시킬 수 있다.  
2. Excutor 프로세스  
- 드라이버 프로세스가 할당한 일을 수행  
- Excutor들이 병렬로 작업할 수 있도록 하는 최소한의 단위인 파티션 존재  
3. 클러스터 매니저  
- 물리적 클러스터 관리

### Spark data structure
1. Spark 대표 데이터 구조
- RDD    
- DataFrame  
- DataSet    
2. RDD  
- 기본  
Spark는 기본적으로 RDD들의 흐름에 의하여 동작한다. RDD는 Reslient Distributed Data로 회복력 있는 분산 데이터로 해석할 수 있다. 이는 RDD의 Lineage 특성 때문이다.  
RDD는 Resilient 즉, 불변의 특성을 가지기에 Read Only이다. 그렇기에 특정 동작을 위해서는 기존 RDD를 변형한 새로운 RDD가 생성되게 되는데, 이 때 생성되는 연산 순서가 바로 Lineage이다.  
이때 하나의 RDD를 노드로 본다면, 노드 간에는 순환이 없으며, 일정한 방향성을 가지게 된다. 이로 인하여 노드 간에는 의존성이 있으며, 노드 간의 순서가 중요해진다. 따라서 노드 간 그래프를 복기하여 다시 계산하는 과정을 통하여 복구가 가능하다.
- RDD 동작 원리  
모든 작업은 새로운 RDD를 생성 혹은 이미 존재하는 RDD를 새로운 RDD로 변형 혹은 최종 결과 계산을 위하여 RDD에서 연산 API를 호출하는 것 중 하나에 의하여 수행된다.  
RDD 동작 원리의 핵심은 Lazy Evaluation이다. 즉, 즉시 실행하지 않는 것으로 Action 연산자를 만나기 전까지는, Transformation 연산자가 아무리 쌓여 있어도 처리하지 않는다.  
Transformation 연산자는 기존 RDD에서 새로운 RDD를 생성하는 동작으로, 리턴 값이 RDD이다. 주요 연산자로는 map(), filter(), union() 존재  
Action 연산자는 기록된 모든 작업을 실제 수행하는 연산자로, 리턴 값이 데이터 또는 실행 결과이다. 주요 연산자로는 collect(), count(), top(num) 존재  
3. Dataframe(SchemaRDD)  
Spark의 DataFrame은 행과 열로 구성된 데이터 분산 collection으로 아래와 같은 특징이 있다.
- 구조화된 데이터 구조: DataFrame은 구조화된 데이터를 다루기 쉽게 하기 위하여 만들어진 데이터 구조로, Spark 사용자는 SparkSQL 등을 통하여 구조화된 데이터의 쿼리를 처리할 수 있다.  
- Garbage Collection 오버헤드 감소: RDD는 데이터를 메모리에 저장하지만, DataFrame은 데이터를 off-heap(RAM 영역)영역에 저장한다. 이를 통하여 Garbage Collection 오버헤드를 감소시킬 수 있다.
- 직렬화 오버헤드 감소: DataFrame은 off-heap 메모리를 사용한 직렬화를 통하여 오버헤드를 크게 감소
- Flexibility & Scalability: DataFrame은 CSV, Cassandra 등 다양한 형태의 데이터를 직접 지원한다.  
4. Dataset  
- 여러 DataFrame을 담아두는 공간으로, 여러개의 DataFrame이 Dataset 내에 저장된다.  


Ref: https://www.kaggle.com/code/tientd95/pyspark-for-data-science

In [ ]:
!pip install pyspark

# Library Load & Data Load

In [ ]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import random
import os
import warnings
warnings.filterwarnings('ignore')

# spark 라이브러리
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql import SQLContext
from pyspark.sql.functions import mean,col,split,regexp_extract, when, lit,max,min
# col: 주어진 column name에 해당하는 컬럼을 데이터에서 반환
# split: str을 주어진 패턴에 맞춰 split 한다
# when: 주어진 조건 list와 일치하는 여러 결과 식 중 하나를 반환
# lit: literal 값의 열을 생성
from pyspark.ml.feature import StringIndexer, VectorAssembler
# StringIndexer: string column of label to an ML column of label indice로 mapping
# VectorAssembler: multiple column을 병합하여 하나의 vector column으로 변환
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import QuantileDiscretizer
# QuantileDiscretizer: 연속형 변수의 열을 가져와서, bin값 만큼의 범주 형 값의 열을 출력
from pyspark.ml.stat import Correlation

In [ ]:
#SparkSession 생성 시 아래와 같이 입력할 수 있는데, 이는 CPU 코어 개수 혹은 클러스터 매니저를 명시할 수 있도록 해준다
# master("local[2]") == CPU 코어 2개
# spark = SparkSession.builder.master("local[2]").appName("Linear-Regression-California-Housing").getOrCreate()

In [ ]:
#SparkSession 생성
spark = SparkSession.builder.appName('recommender_system').getOrCreate()

In [ ]:
# load the dataset and create spark dataframe
train_df =  spark.read.csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv',inferSchema=True,header=True)
test_df = spark.read.csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv',inferSchema=True,header=True)

# limit 5, like pandas head function
# toPandas make Pyspark DataFrame as Pandas table
train_df.limit(5).toPandas()

In [ ]:
train_df.describe().toPandas()

Data fields
Here's a brief version of what you'll find in the data description file.

- SalePrice - the property's sale price in dollars. This is the target variable that you're trying to predict.
- MSSubClass: The building class
- MSZoning: The general zoning classification
- LotFrontage: Linear feet of street connected to property
- LotArea: Lot size in square feet
- Street: Type of road access
- Alley: Type of alley access
- LotShape: General shape of property
- LandContour: Flatness of the property
- Utilities: Type of utilities available
- LotConfig: Lot configuration
- LandSlope: Slope of property
- Neighborhood: Physical locations within Ames city limits
- Condition1: Proximity to main road or railroad
- Condition2: Proximity to main road or railroad (if a second is present)
- BldgType: Type of dwelling
- HouseStyle: Style of dwelling
- OverallQual: Overall material and finish quality
- OverallCond: Overall condition rating
- YearBuilt: Original construction date
- YearRemodAdd: Remodel date
- RoofStyle: Type of roof
- RoofMatl: Roof material
- Exterior1st: Exterior covering on house
- Exterior2nd: Exterior covering on house (if more than one material)
- MasVnrType: Masonry veneer type
- MasVnrArea: Masonry veneer area in square feet
- ExterQual: Exterior material quality
- ExterCond: Present condition of the material on the exterior
- Foundation: Type of foundation
- BsmtQual: Height of the basement
- BsmtCond: General condition of the basement
- BsmtExposure: Walkout or garden level basement walls
- BsmtFinType1: Quality of basement finished area
- BsmtFinSF1: Type 1 finished square feet
- BsmtFinType2: Quality of second finished area (if present)
- BsmtFinSF2: Type 2 finished square feet
- BsmtUnfSF: Unfinished square feet of basement area
- TotalBsmtSF: Total square feet of basement area
- Heating: Type of heating
- HeatingQC: Heating quality and condition
- CentralAir: Central air conditioning
- Electrical: Electrical system
- 1stFlrSF: First Floor square feet
- 2ndFlrSF: Second floor square feet
- LowQualFinSF: Low quality finished square feet (all floors)
- GrLivArea: Above grade (ground) living area square feet
- BsmtFullBath: Basement full bathrooms
- BsmtHalfBath: Basement half bathrooms
- FullBath: Full bathrooms above grade
- HalfBath: Half baths above grade
- Bedroom: Number of bedrooms above basement level
- Kitchen: Number of kitchens
- KitchenQual: Kitchen quality
- TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)
- Functional: Home functionality rating
- Fireplaces: Number of fireplaces
- FireplaceQu: Fireplace quality
- GarageType: Garage location
- GarageYrBlt: Year garage was built
- GarageFinish: Interior finish of the garage
- GarageCars: Size of garage in car capacity
- GarageArea: Size of garage in square feet
- GarageQual: Garage quality
- GarageCond: Garage condition
- PavedDrive: Paved driveway
- WoodDeckSF: Wood deck area in square feet
- OpenPorchSF: Open porch area in square feet
- EnclosedPorch: Enclosed porch area in square feet
- 3SsnPorch: Three season porch area in square feet
- ScreenPorch: Screen porch area in square feet
- PoolArea: Pool area in square feet
- PoolQC: Pool quality
- Fence: Fence quality
- MiscFeature: Miscellaneous feature not covered in other categories
- MiscVal: Value of miscellaneous feature
- MoSold: Month Sold
- YrSold: Year Sold
- SaleType: Type of sale
- SaleCondition: Condition of sale

In [ ]:
# withColumn을 활용하여 DataFrame의 값을 변경할 수 있다.
# 이때,  when을 통하여 condition based 하게 값을 변경할 수 있다.
train_df = train_df.withColumn("SaleCondition",when(train_df.SaleCondition == "Normal",0).when(train_df.SaleCondition == "Abnorml",1))
train_df.limit(5).toPandas()

In [ ]:
# printSchema()를 사용하여 feature별 data type을 확인할 수 있다.
# pandas의 info()와 유사하다고 생각된다
train_df.printSchema()

In [ ]:
# get Dataframe field
print("list of Dataframe filed")
print(train_df.schema.fields)

In [ ]:
# get Dataframe 1st filed's name
print("Dataframe 1st filed's name")
print(train_df.schema.fields[0].name)

# EDA using PySpark

In [ ]:
from pyspark.sql.types import StringType
# Check DataFrame's column dataType is string
cols = [f.name for f in train_df.schema.fields if isinstance(f.dataType, StringType)]

# Drop String type column
train_df.drop(*cols).printSchema()

In [ ]:
# Detect Missing Value
columns = train_df.columns
missing_values = {}
for idx, column in enumerate(columns):
    missing_count = train_df.where(col(column).isin([0,None,np.nan])).count()
    missing_values.update({column:missing_count})
missing_df = pd.DataFrame.from_dict([missing_values])

In [ ]:
missing_df

In [ ]:
traind_df = train_df.fillna(0)
train_df.printSchema()

### Numerical variable Targeting

In [ ]:
# Visualization SalePrice
# fig는 데이터가 담기는 프레임으로, 그래프가 그려질 액자이다.
# ax는 실제 데이터가 그려지는 캔버스이다.
# 인자 중 figaspect는 width,height의 ratio
plt.figure(figsize=(15,10))
fig,ax = plt.subplots(nrows=1,ncols=1,figsize=plt.figaspect(0.25))

sale_price = train_df.toPandas()[["SalePrice","Id"]]
sns.lineplot(y="SalePrice",x="Id",data=sale_price, ax=ax)

In [ ]:
# 연도별 판매가 평균 
train_df.groupby("YrSold").agg(mean("SalePrice")).toPandas()

In [ ]:
plt.figure(figsize=(15,10))
fig,ax = plt.subplots(nrows=1,ncols=1,figsize=plt.figaspect(0.25))

sale_price_byYear = train_df.groupby("YrSold").agg(mean("SalePrice")).toPandas()
sns.lineplot(y="avg(SalePrice)",x="YrSold",data=sale_price_byYear, ax=ax)

집값은 하락하는 추세로 보이나, 기타 요소를 반영하여 상세 분석 필요

In [ ]:
train_df.groupby("YearBuilt").count().sort("YearBuilt").show()

In [ ]:
plt.figure(figsize=(15,10))
fig,ax = plt.subplots(nrows=1,ncols=1,figsize=plt.figaspect(0.25))

sale_price_byYear = train_df.groupby("YearBuilt").agg(mean("SalePrice")).toPandas()
sns.lineplot(y="avg(SalePrice)",x="YearBuilt",data=sale_price_byYear, ax=ax)

In [ ]:
train_df.filter(train_df.YrSold == 2007).toPandas()

In [ ]:
# Use filter for condition search
# filter the rows from RDD/DataFrame based on given condition
# pandas way is train_df[train_df["YrSold"]==2006]
train_df.filter(train_df.YrSold == 2006).groupby("YearBuilt").agg(mean("SalePrice")).sort("YearBuilt").toPandas()

In [ ]:
# Avg SalePrice by Built Year

plt.figure(figsize=(18,12))
fig,ax = plt.subplots(nrows=3,ncols=2,figsize=plt.figaspect(0.25))
fig.suptitle("AVG SalePrice with Built year")

sale_price_byYear = train_df.filter(train_df.YrSold == 2006).groupby("YearBuilt").agg(mean("SalePrice")).toPandas()
sns.lineplot(y="avg(SalePrice)",x="YearBuilt",data=sale_price_byYear, ax=ax[0][0])

sale_price_byYear = train_df.filter(train_df.YrSold == 2007).groupby("YearBuilt").agg(mean("SalePrice")).toPandas()
sns.lineplot(y="avg(SalePrice)",x="YearBuilt",data=sale_price_byYear, ax=ax[0][1])

sale_price_byYear = train_df.filter(train_df.YrSold == 2008).groupby("YearBuilt").agg(mean("SalePrice")).toPandas()
sns.lineplot(y="avg(SalePrice)",x="YearBuilt",data=sale_price_byYear, ax=ax[1][0])

sale_price_byYear = train_df.filter(train_df.YrSold == 2009).groupby("YearBuilt").agg(mean("SalePrice")).toPandas()
sns.lineplot(y="avg(SalePrice)",x="YearBuilt",data=sale_price_byYear, ax=ax[1][1])

sale_price_byYear = train_df.filter(train_df.YrSold == 2010).groupby("YearBuilt").agg(mean("SalePrice")).toPandas()
sns.lineplot(y="avg(SalePrice)",x="YearBuilt",data=sale_price_byYear, ax=ax[2][0])

구축 주택 중 가격이 높은 사례가 일부 존재, Outlier로 판단할 수 있을 듯 하다

In [ ]:
# 이상 데이터라 추측되는 값 조회
train_df.filter(train_df.YearBuilt <= 1900).filter(train_df.SalePrice > 200000).groupby("YearBuilt").agg(mean("SalePrice")).sort("YearBuilt").toPandas()

In [ ]:
train_df_abnormal = train_df.filter(train_df.YearBuilt <= 1900).filter(train_df.SalePrice > 200000)
train_df_abnormal.limit(3).toPandas()

In [ ]:
train_df_abnormal.drop(*cols).toPandas()

In [ ]:
plt.figure(figsize=(20,15))
sns.heatmap(train_df_abnormal.drop(*cols).toPandas().corr(),annot=True)

SalePrice에 영향을 줄 수 있을 후보군
- LotArea  
- OverallQual  
- OverallCond  
- LowQualFinSF  
- TotRmsAbvGrd  
- Fireplaces  
- GarageARea  
- WoddDeckSF  
- ScreenPorch  

In [ ]:
train_df.filter(train_df.YearBuilt == 1892).toPandas()[['LotArea','OverallQual','OverallCond','LowQualFinSF','TotRmsAbvGrd','Fireplaces','GarageArea',
                                                       'WoodDeckSF','ScreenPorch','SalePrice']].sort_values('SalePrice')

In [ ]:
train_df.filter(train_df.YearBuilt == 1880).toPandas()[['LotArea','OverallQual','OverallCond','LowQualFinSF','TotRmsAbvGrd','Fireplaces','GarageArea',
                                                       'WoodDeckSF','ScreenPorch','SalePrice']].sort_values('SalePrice')

In [ ]:
train_df.filter(train_df.YearBuilt.isin([1880,1892,1893])).toPandas()[['LotArea','OverallQual','OverallCond','LowQualFinSF','TotRmsAbvGrd','Fireplaces','GarageArea',
                                                       'WoodDeckSF','ScreenPorch','SalePrice']].sort_values('SalePrice')

후보군의 값이 커짐에 따라 SalePrice가 올라가는 경향이 있다.

In [ ]:
train_df.filter(train_df.YearBuilt.isin([1880,1892,1893])).toPandas()[['YearBuilt','YearRemodAdd','SalePrice']].sort_values('SalePrice')

In [ ]:
# Add bulit_remod column and append new data(between built and remodeling)
train_df = train_df.withColumn('Built_Remod', train_df.YearRemodAdd - train_df.YearBuilt)
train_df.toPandas()

In [ ]:
# 구축 후 리모델링까지의 기간 확인
train_df.groupBy("Built_Remod").count().orderBy("Built_Remod").toPandas()

In [ ]:
train_df_remodeling_info = train_df.groupBy("Built_Remod").count().orderBy("Built_Remod").toPandas()
sns.barplot(x="Built_Remod",y="count" ,data=train_df_remodeling_info)

다수의 주택이 Remodeling을 하지 않은 것을 확인  
아래 두 가지 feature engineering 활동 필요  
- Remodeling 유/무 변수 추가
- Remodeling 연도 변수 제거

In [ ]:
train_df.filter(train_df.YearBuilt.isin([1880,1892,1893])).toPandas()[['Built_Remod','SalePrice']].sort_values('SalePrice')

In [ ]:
train_df.toPandas()[['Built_Remod','SalePrice']].sort_values('SalePrice')

In [ ]:
# SalePrice with year between built and remodeling
plt.figure(figsize=(15,10))
fig,ax = plt.subplots(nrows=1,ncols=1,figsize=plt.figaspect(0.25))
fig.suptitle("SalePrice with year between built and remodeling")

sale_price_byBuiltRemod = train_df.toPandas()[['Built_Remod','SalePrice']].sort_values('SalePrice')
sns.lineplot(y="SalePrice",x="Built_Remod",data=sale_price_byBuiltRemod, ax=ax)

remodeling까지의 기간이 60년을 넘기는 구간부터 remodeling에 따른 비용 상승의 폭이 큰 것을 확인할 수 있다.  
이를 바탕으로, 구(舊)축을 remodeling할 경우 비용 상승 폭이 훨씬 큰 것이라 추측

In [ ]:
print("original")
print(train_df.count())

print("-"*10)

print("Abnormal")
print(train_df.filter(train_df.YearBuilt <= 1900).filter(train_df.SalePrice > 200000).count())

In [ ]:
# 이상값이라 판단한 기준 외의 값 추출
cond = (col('YearBuilt') <= 1900) & (col('SalePrice') > 200000)

print("After remove abnormal")
print(train_df.filter(~cond).count())
train_df.filter(~cond).toPandas()

In [ ]:
train_df_preprocess = train_df.filter(~cond)

In [ ]:
# Avg SalePrice by Built Year after remove abnormal
plt.figure(figsize=(18,12))
fig,ax = plt.subplots(nrows=3,ncols=2,figsize=plt.figaspect(0.25))
fig.suptitle("Avg SalePrice with Built year after remove abnormal")

sale_price_byYear = train_df_preprocess.filter(train_df_preprocess.YrSold == 2006).groupby("YearBuilt").agg(mean("SalePrice")).toPandas()
sns.lineplot(y="avg(SalePrice)",x="YearBuilt",data=sale_price_byYear, ax=ax[0][0])

sale_price_byYear = train_df_preprocess.filter(train_df_preprocess.YrSold == 2007).groupby("YearBuilt").agg(mean("SalePrice")).toPandas()
sns.lineplot(y="avg(SalePrice)",x="YearBuilt",data=sale_price_byYear, ax=ax[0][1])

sale_price_byYear = train_df_preprocess.filter(train_df_preprocess.YrSold == 2008).groupby("YearBuilt").agg(mean("SalePrice")).toPandas()
sns.lineplot(y="avg(SalePrice)",x="YearBuilt",data=sale_price_byYear, ax=ax[1][0])

sale_price_byYear = train_df_preprocess.filter(train_df_preprocess.YrSold == 2009).groupby("YearBuilt").agg(mean("SalePrice")).toPandas()
sns.lineplot(y="avg(SalePrice)",x="YearBuilt",data=sale_price_byYear, ax=ax[1][1])

sale_price_byYear = train_df_preprocess.filter(train_df_preprocess.YrSold == 2010).groupby("YearBuilt").agg(mean("SalePrice")).toPandas()
sns.lineplot(y="avg(SalePrice)",x="YearBuilt",data=sale_price_byYear, ax=ax[2][0])

In [ ]:
plt.figure(figsize=(15,10))
fig,ax = plt.subplots(nrows=1,ncols=1,figsize=plt.figaspect(0.25))

sale_price_byYear = train_df_preprocess.groupby("YearBuilt").agg(mean("SalePrice")).toPandas()
sns.lineplot(y="avg(SalePrice)",x="YearBuilt",data=sale_price_byYear, ax=ax)

신축으로 갈 수록 주택 가격이 증가하는 경향이 나타남

In [ ]:
# Mean SalePrice by Year
plt.figure(figsize=(15,10))
fig,ax = plt.subplots(nrows=1,ncols=1,figsize=plt.figaspect(0.25))

sale_price_byYear = train_df_preprocess.groupby("YrSold").agg(mean("SalePrice")).toPandas()
sns.lineplot(y="avg(SalePrice)",x="YrSold",data=sale_price_byYear, ax=ax)

특정 연도에 일부 비싼 집들(비교적 신축에서 가격이 높은 경향이 보인다)이 팔렸다고 추측  

In [ ]:
plt.figure(figsize=(15,10))
fig,ax = plt.subplots(nrows=1,ncols=2,figsize=plt.figaspect(0.25))

# Max SalePrice by Year
sale_price_byYear = train_df_preprocess.groupby("YrSold").agg(max("SalePrice")).toPandas()
sns.lineplot(y="max(SalePrice)",x="YrSold",data=sale_price_byYear, ax=ax[0])
ax[0].set_title("Max SalePrice by Year")

# Min SalePrice by Year
sale_price_byYear = train_df_preprocess.groupby("YrSold").agg(min("SalePrice")).toPandas()
sns.lineplot(y="min(SalePrice)",x="YrSold",data=sale_price_byYear, ax=ax[1])
ax[1].set_title("Min SalePrice by Year")

Max 값의 영향도가 큰 것을 파악

In [ ]:
train_df.select('SalePrice').toPandas().boxplot()

In [ ]:
train_df.toPandas().boxplot(column=['SalePrice', 'YrSold'],by=['YrSold'])
plt.tight_layout()

SalePrice 값에 이상치(비싼 집)가 존재하는 것 확인

In [ ]:
'''
Q3 : 100개의 데이터로 가정 시, 25번째로 높은 값
Q1 : 100개의 데이터로 가정 시, 75번째로 높은 값
IQR : Q3 - Q1의 차이
이상치 : Q3 + 1.5 * IQR < Value or Value < Q1 - 1.5 * IQR 
Quantil 함수를 통하여 백분위 계산 진행
'''
Q3 = train_df.select("SalePrice").toPandas().quantile(.75)
Q1 = train_df.select("SalePrice").toPandas().quantile(.25)
IQR = Q3 - Q1

In [ ]:
# divide outlier data and normal data
# compare A group(high price house) vs B group(normal price house)
train_pd = train_df.toPandas()
outlier_condition = Q3 + (1.5*IQR)
high_value_house = train_pd[train_pd["SalePrice"] > outlier_condition[0]]
normal_value_house = train_pd[train_pd["SalePrice"] <= outlier_condition[0]]

In [ ]:
high_value_house.dropna(inplace=True)
normal_value_house.dropna(inplace=True)

In [ ]:
high_value_house.head()

In [ ]:
normal_value_house.head()

In [ ]:
high_value_house[high_value_house["LotFrontage"]=="NA"]

In [ ]:
# Group compare (A vs B)
# 'OverallCond' feature has no any different
print("High Price")
print(high_value_house[high_value_house["LotFrontage"]!="NA"][['LotFrontage','LotArea','OverallQual','LowQualFinSF','TotRmsAbvGrd','Fireplaces','GarageArea',
                                                       'WoodDeckSF','ScreenPorch','SalePrice']].astype(int).describe())

print("-"*70)

print("Normal Price")
print(normal_value_house[normal_value_house["LotFrontage"]!="NA"][['LotFrontage','LotArea','OverallQual','LowQualFinSF','TotRmsAbvGrd','Fireplaces','GarageArea',
                                                       'WoodDeckSF','ScreenPorch','SalePrice']].astype(int).describe())

In [ ]:
plt.figure(figsize=(15,10))
fig,ax = plt.subplots(nrows=1,ncols=1,figsize=plt.figaspect(0.45))

sale_condition = train_df.groupby("SaleCondition").count().toPandas()
ax.pie(x="count",labels="SaleCondition",data=sale_condition, autopct='%1.1f%%', shadow=True,startangle=90
    , radius=1)

In [ ]:
print("High price house sale condition")
print(high_value_house.groupby("SaleCondition").count()["SalePrice"])
print("-"*50)
print("low price house sale condition")
print(normal_value_house.groupby("SaleCondition").count()["SalePrice"])

### Categorical variable Targeting

In [ ]:
# toPandas로 pandas DataFrame으로 변경 후 분석
print("Pandas Style")
print(train_df.toPandas()["Street"].value_counts())

print("-"*10)

# Spark DataFrame 그대로 분석
print("Spark Style")
print(train_df.groupBy('Street').count().show())

In [ ]:
print(train_df.groupBy("HouseStyle").count().show())

In [ ]:
print(train_df.groupBy("SaleType").count().show())

In [ ]:
train_df.groupBy("SaleType").count().toPandas()

In [ ]:
categorical_df = train_df.groupBy("SaleType").count().toPandas()
sns.barplot(x="SaleType",y="count",data=categorical_df)

In [ ]:
columnList = [item[0] for item in train_df.dtypes if item[1].startswith('string')]
len(columnList)

In [ ]:
columnList = [item[0] for item in train_df.dtypes if item[1].startswith('string')]

fig,ax = plt.subplots(nrows=9,ncols=5,figsize=(22,20),constrained_layout=True)
fig.subplots_adjust(hspace=2)

idx, jdx = 0, 0
for arg in columnList:
    categorical_df = train_df.groupBy(arg).count().toPandas()
    sns.barplot(x=arg,y="count",data=categorical_df,ax=ax[jdx][idx])
    ax[jdx,idx].set_title(arg)
    if idx == 4:
        jdx += 1
        idx = 0
    else:
        idx +=1

street, PoolQC, Heating, BsmtFinType2, Condition2,RoofMati,Utilites, Alley 변수는 영향도가 미미할 것 이라 추측